In [1]:
import pandas as pd

import eval_fns

# Setup

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Set your root directory below. Make sure the `/data`, `/recommendations`, and `/data_exports` folders are uploaded/generated by the previous steps and are situated in this directory.

In [ ]:
# Adjust your root directory
root = '/content/drive/MyDrive/KuaiRec/'

# Loading the data

In [ ]:
# root = '../'

prediction_scores_caption = pd.read_csv(root + 'recommendations/recommendations_caption_test_full.csv')
prediction_scores_ncf = pd.read_csv(root + 'recommendations/final_w_clustering_batch_size512_num_epochs20_lr0_001_embedding.csv')
prediction_scores_hybrid = pd.read_csv(root + 'recommendations/recommendations_hybrid_test_full.csv')
prediction_scores_random = pd.read_csv(root + 'recommendations/recommendations_random_test_full.csv')

prediction_scores_caption_val = pd.read_csv(root + 'recommendations/recommendations_caption_val_full.csv')
prediction_scores_ncf_val = pd.read_csv(root + 'recommendations/w_clustering_batch_size512_num_epochs20_lr0_001_embedding_dim64.csv')
prediction_scores_hybrid_val = pd.read_csv(root + 'recommendations/recommendations_hybrid_val_full.csv')
prediction_scores_random_val = pd.read_csv(root + 'recommendations/recommendations_random_val_full.csv')

joined_train_data = pd.read_csv(root + 'data_exports/joined_train_data.csv')
joined_val_data = pd.read_csv(root + 'data_exports/joined_val_data.csv')
joined_test_data = pd.read_csv(root + 'data_exports/joined_test_data.csv')

joined_train_val_data = pd.concat([joined_train_data, joined_val_data])

video_data = pd.read_csv(root + 'data/kuairec_caption_category_translated.csv', index_col=0)

# Data preprocessing

In [3]:
# Rename
prediction_scores_caption = prediction_scores_caption.rename(columns={'watch_ratio': 'predicted_watch_ratio'})
prediction_scores_ncf = prediction_scores_ncf.rename(columns={'watch_ratio': 'predicted_watch_ratio'})
prediction_scores_hybrid = prediction_scores_hybrid.rename(columns={'watch_ratio': 'predicted_watch_ratio'})
prediction_scores_random = prediction_scores_random.rename(columns={'watch_ratio': 'predicted_watch_ratio'})

prediction_scores_caption_val = prediction_scores_caption_val.rename(columns={'watch_ratio': 'predicted_watch_ratio'})
prediction_scores_ncf_val = prediction_scores_ncf_val.rename(columns={'watch_ratio': 'predicted_watch_ratio'})
prediction_scores_hybrid_val = prediction_scores_hybrid_val.rename(columns={'watch_ratio': 'predicted_watch_ratio'})
prediction_scores_random_val = prediction_scores_random_val.rename(columns={'watch_ratio': 'predicted_watch_ratio'})

# Sort predictions
prediction_scores_caption = prediction_scores_caption.sort_values(by=['user_id', 'video_id'])
prediction_scores_ncf = prediction_scores_ncf.sort_values(by=['user_id', 'video_id'])
prediction_scores_hybrid = prediction_scores_hybrid.sort_values(by=['user_id', 'video_id'])
prediction_scores_random = prediction_scores_random.sort_values(by=['user_id', 'video_id'])

prediction_scores_caption_val = prediction_scores_caption_val.sort_values(by=['user_id', 'video_id'])
prediction_scores_ncf_val = prediction_scores_ncf_val.sort_values(by=['user_id', 'video_id'])
prediction_scores_hybrid_val = prediction_scores_hybrid_val.sort_values(by=['user_id', 'video_id'])
prediction_scores_random_val = prediction_scores_random_val.sort_values(by=['user_id', 'video_id'])

prediction_scores_caption

,Unnamed: 0,user_id,video_id,predicted_watch_ratio
0,0,14,0,4.727335
1,1,14,1,4.573298
2,2,14,2,4.747150
3,3,14,3,4.663642
4,4,14,4,4.555126
...,...,...,...,...
15137203,15137203,7162,10723,4.788451
15137204,15137204,7162,10724,4.616125
15137205,15137205,7162,10725,4.622754
15137206,15137206,7162,10726,4.632048


## Get user watch history

We want to be able to filter out videos that the user has already watched. This is so that we recommend new videos instead.

In [4]:
user_watch_history = eval_fns.get_user_watch_history(joined_train_val_data)

user_watch_history_val = eval_fns.get_user_watch_history(joined_train_data)

## Getting ground truth videos for each user

In [5]:
users_in_train_val_data = set(joined_train_val_data['user_id'])
videos_in_train_val_data = set(joined_train_val_data['video_id'])

ground_truth = eval_fns.get_ground_truth(joined_test_data[['user_id', 'video_id', 'watch_ratio']], users_in_train_val_data, videos_in_train_val_data, user_watch_history)

users_in_train_data = set(joined_train_data['user_id'])
videos_in_train_data = set(joined_train_data['video_id'])

ground_truth_val = eval_fns.get_ground_truth(joined_val_data[['user_id', 'video_id', 'watch_ratio']], users_in_train_data, videos_in_train_data, user_watch_history_val)

In [6]:
# Ground truth scores for user 14
ground_truth[ground_truth['user_id'] == 14]

,user_id,video_id,watch_ratio
27,14,4184,3.234123
0,14,6293,2.442865
184,14,5954,1.899621
31,14,10354,1.884053
18,14,1352,1.780083
...,...,...,...
169,14,6270,0.062602
113,14,7736,0.060968
116,14,10140,0.032761
183,14,2755,0.032283


## Getting recommendations for each user

In [7]:
videos_in_test_data = set(joined_test_data['video_id'])

recommendations_caption = eval_fns.get_user_recommendations(prediction_scores_caption, videos_in_test_data, user_watch_history)
recommendations_ncf = eval_fns.get_user_recommendations(prediction_scores_ncf, videos_in_test_data, user_watch_history)
recommendations_hybrid = eval_fns.get_user_recommendations(prediction_scores_hybrid, videos_in_test_data, user_watch_history)
recommendations_random = eval_fns.get_user_recommendations(prediction_scores_random, videos_in_test_data, user_watch_history)

videos_in_val_data = set(joined_val_data['video_id'])

recommendations_caption_val = eval_fns.get_user_recommendations(prediction_scores_caption_val, videos_in_val_data, user_watch_history_val)
recommendations_ncf_val = eval_fns.get_user_recommendations(prediction_scores_ncf_val, videos_in_val_data, user_watch_history_val)
recommendations_hybrid_val = eval_fns.get_user_recommendations(prediction_scores_hybrid_val, videos_in_val_data, user_watch_history_val)
recommendations_random_val = eval_fns.get_user_recommendations(prediction_scores_random_val, videos_in_val_data, user_watch_history_val)

100%|██████████| 1411/1411 [00:03<00:00, 426.92it/s]


In [8]:
# Recommendations based on caption for user 14
recommendations_caption[recommendations_caption['user_id'] == 14]

,Unnamed: 0,user_id,video_id,predicted_watch_ratio
1268,1268,14,1268,4.801478
1353,1353,14,1353,4.799540
8693,8693,14,8693,4.791233
3189,3189,14,3189,4.788725
3725,3725,14,3725,4.788245
...,...,...,...,...
4676,4676,14,4676,4.327762
3131,3131,14,3131,4.319678
6287,6287,14,6287,4.315479
4358,4358,14,4358,4.278086


In [9]:
reco_grp_caption = recommendations_caption.groupby('user_id')
reco_grp_ncf = recommendations_ncf.groupby('user_id')
reco_grp_hybrid = recommendations_hybrid.groupby('user_id')
reco_grp_random = recommendations_random.groupby('user_id')

reco_grp_caption_val = recommendations_caption_val.groupby('user_id')
reco_grp_ncf_val = recommendations_ncf_val.groupby('user_id')
reco_grp_hybrid_val = recommendations_hybrid_val.groupby('user_id')
reco_grp_random_val = recommendations_random_val.groupby('user_id')

ground_truth_grp = ground_truth.groupby('user_id')
ground_truth_grp_val = ground_truth_val.groupby('user_id')

In [10]:
k = 100

# Get top 100 ground truth and recommendations for user 14
top_100_ground_truth_user_14 = eval_fns.get_top_k_for_user(k, 14, ground_truth)
top_100_recommendations_user_14_caption = eval_fns.get_top_k_for_user(k, 14, reco_grp_caption)
top_100_recommendations_user_14_ncf = eval_fns.get_top_k_for_user(k, 14, reco_grp_ncf)
top_100_recommendations_user_14_hybrid = eval_fns.get_top_k_for_user(k, 14, reco_grp_hybrid)
top_100_recommendations_user_14_random = eval_fns.get_top_k_for_user(k, 14, reco_grp_random)

# top_100_ground_truth_user_14
top_100_recommendations_user_14_caption

,Unnamed: 0,user_id,video_id,predicted_watch_ratio
1268,1268,14,1268,4.801478
1353,1353,14,1353,4.799540
8693,8693,14,8693,4.791233
3189,3189,14,3189,4.788725
3725,3725,14,3725,4.788245
...,...,...,...,...
4791,4791,14,4791,4.716323
1368,1368,14,1368,4.716151
9142,9142,14,9142,4.715870
6441,6441,14,6441,4.715367


## Calculation of Evaluation Metrics

### Category-Aware NDCG@k

In [11]:
# Get the category-aware NDCG@100 for user 14
print(eval_fns.get_category_ndcg_at_k(top_100_recommendations_user_14_caption, top_100_ground_truth_user_14, video_data))
print(eval_fns.get_category_ndcg_at_k(top_100_recommendations_user_14_ncf, top_100_ground_truth_user_14, video_data))
print(eval_fns.get_category_ndcg_at_k(top_100_recommendations_user_14_hybrid, top_100_ground_truth_user_14, video_data))
print(eval_fns.get_category_ndcg_at_k(top_100_recommendations_user_14_random, top_100_ground_truth_user_14, video_data))

0.9166669994041701
0.9714503235147487
0.9676650456458153
0.9638014349334976


### Distinct Categories @ k

In [12]:
# Get Distinct Categories @ 100 for user 14
print(eval_fns.get_user_distinct_categories_at_k(k, 14, recommendations_caption, video_data))
print(eval_fns.get_user_distinct_categories_at_k(k, 14, recommendations_ncf, video_data))
print(eval_fns.get_user_distinct_categories_at_k(k, 14, recommendations_hybrid, video_data))
print(eval_fns.get_user_distinct_categories_at_k(k, 14, recommendations_random, video_data))

22
24
24
26


### Average watch ratio @ k

In [13]:
# Get avg watch ratio @ 100 for user 14
print(eval_fns.get_user_avg_watch_ratio_at_k(k, 14, reco_grp_caption, 'watch_ratio', ground_truth_grp))
print(eval_fns.get_user_avg_watch_ratio_at_k(k, 14, reco_grp_ncf, 'watch_ratio', ground_truth_grp))
print(eval_fns.get_user_avg_watch_ratio_at_k(k, 14, reco_grp_hybrid, 'watch_ratio', ground_truth_grp))
print(eval_fns.get_user_avg_watch_ratio_at_k(k, 14, reco_grp_random, 'watch_ratio', ground_truth_grp))

0.7006994420168466
0.8310018606277393
0.8084503473417523
0.7433988546754337


### Precision@k, Recall@k, F1Score@k

In [16]:
threshold = 0.5

# Precision, recall, and F1 @ 100 for user 14
precision, recall, f1 = eval_fns.get_user_precision_recall_f1_at_k(k, 14, reco_grp_caption, ground_truth, threshold)
precision_ncf, recall_ncf, f1_ncf = eval_fns.get_user_precision_recall_f1_at_k(k, 14, reco_grp_ncf, ground_truth, threshold)
precision_hybrid, recall_hybrid, f1_hybrid = eval_fns.get_user_precision_recall_f1_at_k(k, 14, reco_grp_hybrid, ground_truth, threshold)
precision_random, recall_random, f1_random = eval_fns.get_user_precision_recall_f1_at_k(k, 14, reco_grp_random, ground_truth, threshold)
print(precision, recall, f1)
print(precision_ncf, recall_ncf, f1_ncf)
print(precision_hybrid, recall_hybrid, f1_hybrid)
print(precision_random, recall_random, f1_random)

0.66 0.6470588235294118 0.6534653465346535
0.74 0.7254901960784313 0.7326732673267328
0.76 0.7450980392156863 0.7524752475247525
0.65 0.6372549019607843 0.6435643564356436


# Getting all metrics (Validation)

In [17]:
metrics_df_caption_val = eval_fns.get_all_metrics(k, ground_truth_grp_val, reco_grp_caption_val, video_data, threshold, by_cluster = False)
metrics_df_ncf_val = eval_fns.get_all_metrics(k, ground_truth_grp_val, reco_grp_ncf_val, video_data, threshold, by_cluster = False)
metrics_df_hybrid_val = eval_fns.get_all_metrics(k, ground_truth_grp_val, reco_grp_hybrid_val, video_data, threshold, by_cluster = False)
metrics_df_random_val = eval_fns.get_all_metrics(k, ground_truth_grp_val, reco_grp_random_val, video_data, threshold, by_cluster = False)

100%|██████████| 1411/1411 [00:24<00:00, 56.58it/s]


In [18]:
metrics_combined_val = pd.concat([metrics_df_caption_val, metrics_df_ncf_val, metrics_df_hybrid_val, metrics_df_random_val], axis=0)

metrics_combined_val['recommendation_type'] = ['caption', 'ncf', 'hybrid', 'random']

cols = metrics_combined_val.columns.tolist()
cols = cols[-1:] + cols[:-1]
metrics_combined_val = metrics_combined_val[cols]

metrics_combined_val

,recommendation_type,cluster,NDCG@100,Distinct Categories @ 100,Avg Watch Ratio @ 100,Avg Precision@100,Avg Recall@100,Avg F1@100
0,caption,Overall,0.868736,18.795889,0.862624,0.744486,0.259253,0.378630
0,ncf,Overall,0.950681,24.956768,1.036080,0.836230,0.292687,0.426899
0,hybrid,Overall,0.952720,24.672573,1.034439,0.835159,0.292224,0.426268
0,random,Overall,0.921808,25.933381,0.843575,0.723168,0.252085,0.368097


# Getting all metrics (Test)

In [19]:
metrics_df_caption = eval_fns.get_all_metrics(k, ground_truth_grp, reco_grp_caption, video_data, threshold, by_cluster = False)
metrics_df_ncf = eval_fns.get_all_metrics(k, ground_truth_grp, reco_grp_ncf, video_data, threshold, by_cluster = False)
metrics_df_hybrid = eval_fns.get_all_metrics(k, ground_truth_grp, reco_grp_hybrid, video_data, threshold, by_cluster = False)
metrics_df_random = eval_fns.get_all_metrics(k, ground_truth_grp, reco_grp_random, video_data, threshold, by_cluster = False)

  0%|          | 0/1411 [00:00<?, ?it/s]

100%|██████████| 1411/1411 [00:22<00:00, 61.57it/s]


In [25]:
# concatenate
metrics_combined = pd.concat([metrics_df_caption, metrics_df_ncf, metrics_df_hybrid, metrics_df_random], axis=0)

# add column for the type of recommendation
metrics_combined['recommendation_type'] = ['caption', 'ncf', 'hybrid', 'random']

# make it appear to the left
cols = metrics_combined.columns.tolist()
cols = cols[-1:] + cols[:-1]
metrics_combined = metrics_combined[cols]

metrics_combined

,recommendation_type,cluster,NDCG@100,Distinct Categories @ 100,Avg Watch Ratio @ 100,Avg Precision@100,Avg Recall@100,Avg F1@100
0,caption,Overall,0.907141,21.408930,0.821867,0.700712,0.655970,0.662416
0,ncf,Overall,0.970287,25.734940,0.937810,0.801991,0.747508,0.756924
0,hybrid,Overall,0.962718,25.575478,0.911009,0.771537,0.717137,0.727131
0,random,Overall,0.929594,26.715804,0.797052,0.675538,0.635067,0.639998


# Getting per-cluster performance for ncf

In [26]:
metrics_ncf_per_cluster = eval_fns.get_all_metrics(k, ground_truth, recommendations_ncf, video_data, threshold, by_cluster=True)

100%|██████████| 378/378 [00:10<00:00, 36.40it/s]


In [27]:
metrics_ncf_per_cluster

,cluster,NDCG@100,Distinct Categories @ 100,Avg Watch Ratio @ 100,Avg Precision@100,Avg Recall@100,Avg F1@100
0,0,0.967586,26.230483,0.955919,0.760214,0.736330,0.729868
0,1,0.970797,25.842482,0.941045,0.828275,0.735406,0.761713
0,2,0.970502,25.637681,0.916312,0.794558,0.751553,0.755050
0,3,0.971448,25.351852,0.940958,0.809369,0.765185,0.772579
0,Overall,0.970287,25.734940,0.937810,0.801991,0.747508,0.756924
